In [1]:
import cv2
import numpy as np
import os
import pandas as pd

def compute_glcm(image, distances, angles):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = (gray_image * 255).astype(np.uint8)  # Ensure it's uint8

    # Initialize GLCM for each distance and angle
    glcm_list = []

    for d in distances:
        for a in angles:
            glcm = np.zeros((256, 256), dtype=np.float32)

            # Calculate the offset based on the angle
            if a == 0:  # 0 degrees
                offset = (0, d)
            elif a == np.pi / 4:  # 45 degrees
                offset = (d, d)
            elif a == np.pi / 2:  # 90 degrees
                offset = (d, 0)
            elif a == 3 * np.pi / 4:  # 135 degrees
                offset = (d, -d)

            # Compute GLCM
            for i in range(gray_image.shape[0]):
                for j in range(gray_image.shape[1]):
                    if (i + offset[0] < gray_image.shape[0]) and (j + offset[1] < gray_image.shape[1]):
                        row = gray_image[i, j]
                        col = gray_image[i + offset[0], j + offset[1]]
                        glcm[row, col] += 1

            # Normalize the GLCM
            glcm /= np.sum(glcm)

            # Append the GLCM to the list
            glcm_list.append(glcm)

    return glcm_list

def extract_features_from_glcm(glcm):
    features = []
    for matrix in glcm:
        contrast = np.sum((np.arange(256)[:, None] - np.arange(256)) ** 2 * matrix)
        #dissimilarity = np.sum(np.abs(np.arange(256)[:, None] - np.arange(256)) * matrix)
        homogeneity = np.sum(matrix / (1 + np.abs(np.arange(256)[:, None] - np.arange(256))))
        energy = np.sum(matrix ** 2)
        correlation = np.corrcoef(matrix.flatten())
        
        # Append features to the list
        features.extend([contrast, homogeneity, energy, correlation])

    return features

def process_images(image_folder, distances, angles):
    results = []
    labels = []  # You should define your labels based on your dataset

    for filename in os.listdir(image_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Add more formats if needed
            image_path = os.path.join(image_folder, filename)
            image = cv2.imread(image_path)

            # Compute GLCM
            glcm = compute_glcm(image, distances, angles)

            # Extract features from GLCM
            features = extract_features_from_glcm(glcm)

            # Append features and label to results
            results.append(features)
            labels.append('your_label')  # Replace with actual label logic

    return results, labels

# Specify the folder containing images
image_folder = 'Proposed'  # Replace with your folder path
distances = [1,2,3,4]  # You can adjust distances
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]

# Process the images and extract features
features_results, labels = process_images(image_folder, distances, angles)

# Convert features to a DataFrame
features_data = pd.DataFrame(features_results)

# Add labels to the DataFrame
features_data['label'] = labels

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

# Split the dataset into features and target variable
X = features_data.drop('label', axis=1)  # Features
y = features_data['label']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Naive Bayes classifier
nb_classifier = GaussianNB()

# Train the classifier
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

# Evaluate the classifier
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

  your_label       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

